## Exploración de base de datos lematizada en inglés

In [ ]:
import pandas as pd

def convert_csv_to_parquet(csv_file_path, parquet_file_path):
    # Leer el archivo CSV
    df = pd.read_csv(csv_file_path)
    
    # Guardar el DataFrame en formato Parquet
    df.to_parquet(parquet_file_path, index=False)

# Ruta del archivo CSV de entrada
csv_file_path = '/Users/lcsanchez/Desktop/20230525_tecnoempleo.csv'

# Ruta del archivo Parquet de salida
parquet_file_path = '/Users/lcsanchez/Desktop/20230525_tecnoempleo.parquet'

# Convertir CSV a Parquet
convert_csv_to_parquet(csv_file_path, parquet_file_path)

print(f'Archivo convertido a Parquet y guardado en {parquet_file_path}')


In [ ]:
import pandas as pd

def read_and_verify_parquet(parquet_file_path):
    # Leer el archivo Parquet
    df = pd.read_parquet(parquet_file_path)
    
    # Mostrar la información del DataFrame
    print("Información del DataFrame:")
    print(df.info())
    
    # Mostrar las primeras filas del DataFrame
    print("\nPrimeras filas del DataFrame:")
    print(df.head())

# Ruta del archivo Parquet
parquet_file_path = '/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/parquet/tecnoempleo_lemmatized'

# Leer y verificar el contenido del archivo Parquet
read_and_verify_parquet(parquet_file_path)


In [ ]:
parquet_file_path = '/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/parquet/tecnoempleo_lemmatized'

df = pd.read_parquet(parquet_file_path)

df.columns

In [ ]:
df

In [ ]:
from langdetect import detect

def detect_language(text): 
    '''
    Receives the text and detects its language

    text -> Text to detect the language
    '''
    try: 
        lang = detect(text)

    except: 
        lang = None
    
    return lang


In [ ]:
import pandas as pd

# Supongamos que df es tu DataFrame cargado
# df = pd.read_parquet("path_to_your_parquet_file.parquet")

# Añadir una columna con el idioma detectado
df['language'] = df['description_raw_text'].apply(detect_language)

# Contar las observaciones por idioma
language_counts = df['language'].value_counts()

# Filtrar por español e inglés
spanish_count = language_counts.get('es', 0)  # 'es' para español
english_count = language_counts.get('en', 0)  # 'en' para inglés

print(f"Observaciones en español: {spanish_count}")
print(f"Observaciones en inglés: {english_count}")

# Traducir esta columna

## Traducción de base de datos original (lematizar con idioma unificado)

In [ ]:
import pandas as pd
parquet_file_path = '/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/20230525_tecnoempleo.parquet'
df = pd.read_parquet(parquet_file_path)
df.head(100)

In [ ]:
# Comprobamos qué columnas traducir: ¿title? Sí

# Añadir una columna con el idioma detectado
df['language'] = df['title'].apply(detect_language)

# Contar las observaciones por idioma
language_counts = df['language'].value_counts()

# Filtrar por español e inglés
spanish_count = language_counts.get('es', 0)  # 'es' para español
english_count = language_counts.get('en', 0)  # 'en' para inglés

print(f"Observaciones en español: {spanish_count}")
print(f"Observaciones en inglés: {english_count}")

In [ ]:
df.title[df['language'] = 'es']

In [ ]:
# Comprobamos qué columnas traducir: ¿professionalSkills? Sí

# Añadir una columna con el idioma detectado
df['language'] = df['professionalSkills'].apply(detect_language)

# Contar las observaciones por idioma
language_counts = df['language'].value_counts()

# Filtrar por español e inglés
spanish_count = language_counts.get('es', 0)  # 'es' para español
english_count = language_counts.get('en', 0)  # 'en' para inglés

print(f"Observaciones en español: {spanish_count}")
print(f"Observaciones en inglés: {english_count}")

In [ ]:
# Comprobamos qué columnas traducir: ¿description? Sí

# Añadir una columna con el idioma detectado
df['language'] = df['description'].apply(detect_language)

# Contar las observaciones por idioma
language_counts = df['language'].value_counts()

# Filtrar por español e inglés
spanish_count = language_counts.get('es', 0)  # 'es' para español
english_count = language_counts.get('en', 0)  # 'en' para inglés

print(f"Observaciones en español: {spanish_count}")
print(f"Observaciones en inglés: {english_count}")

In [2]:
import pandas as pd

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
from langdetect import detect
import re


# Initialize the Google Translate API from deep_translator
translator = GoogleTranslator(source='auto', target='es')

# Define the path and read the dataset
path = '/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/'
tecnoempleo = pd.read_parquet(path + '20230525_tecnoempleo.parquet')
# tecnoempleo = tecnoempleo.head(10) #para probar con pocas observaciones

#tecnoempleo_translated = pd.DataFrame(columns=['id', 'title_translated', 'professionalSkills_translated', 'description_translated'])
tecnoempleo_translated = pd.read_parquet('/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/translation/tecnoempleo_translated.parquet')

# IDs to be translated
ids_to_translate = list(set(tecnoempleo['id']) - set(tecnoempleo_translated['id']))
indexes = list(tecnoempleo[tecnoempleo['id'].isin(ids_to_translate)].index)

count = 0

# Helper function to clean summaries (this should be defined based on your specific requirements)
def limpiar_resumenes(text):
    # Eliminar saltos de línea y espacios adicionales
    cleaned_text = text.replace('\n', ' ').strip()
    # Elimina caracteres especiales y números usando expresiones regulares
    cleaned_text = re.sub(r'[^a-zA-ZñáéíóúÁÉÍÓÚ\s]', '', cleaned_text)
    
    return cleaned_text

for i in tqdm(indexes, desc="Procesando ofertas de empleo"):
    count += 1
    id_paper = tecnoempleo['id'][i]
    title = tecnoempleo['title'][i]
    professionalSkills = tecnoempleo['professionalSkills'][i]
    description = tecnoempleo['description'][i]

    # Clean and translate the title
    text_to_translate = limpiar_resumenes(title)
    translated_title = text_to_translate
    if detect(text_to_translate) != 'es':
        translated_title = translator.translate(text_to_translate)

    # Clean and translate professionalSkills
    try:
        if professionalSkills is None:
            translated_professionalSkills = ''
        else:
            text_to_translate = limpiar_resumenes(professionalSkills)
            sentences = [sentence.strip() for sentence in text_to_translate.split('-') if sentence.strip()]
            translated_professionalSkills = []

            for sentence in sentences:
                if detect(sentence) != 'es':
                    translated_professionalSkills.append(translator.translate(sentence))
                else:
                    translated_professionalSkills.append(sentence)

            translated_professionalSkills = ' - '.join(translated_professionalSkills)
    except Exception as e:
        translated_professionalSkills = ''
        print(f"Error processing professionalSkills: {e}")

    # Clean and translate description
    try:
        if description is None:
            translated_description = ''
        else:
            text_to_translate = limpiar_resumenes(description)
            sentences = [sentence.strip() for sentence in text_to_translate.split('.') if sentence.strip()]
            translated_description = []

            for sentence in sentences:
                if detect(sentence) != 'es':
                    translated_description.append(translator.translate(sentence))
                else:
                    translated_description.append(sentence)

            translated_description = '. '.join(translated_description)
    except Exception as e:
        translated_description = ''
        print(f"Error processing description: {e}")

    # Append the translated data to the dataframe
    row = pd.DataFrame([[id_paper, translated_title, translated_professionalSkills, translated_description]], columns=['id', 'title_translated', 'professionalSkills_translated', 'description_translated'])
    tecnoempleo_translated = pd.concat([tecnoempleo_translated, row])

    # Save progress periodically
    if count % 100 == 0:
        tecnoempleo_translated.to_parquet('/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/translation/tecnoempleo_translated.parquet')

# Final save
tecnoempleo_translated.to_parquet('/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/translation/tecnoempleo_translated.parquet')

# Merge the translated data with the original data
df = pd.merge(tecnoempleo, tecnoempleo_translated, on='id')
df.to_parquet('/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/translation/complete_tecnoempleo_spanish.parquet')


Procesando ofertas de empleo:   1%|     | 1298/96902 [19:57<15:44:22,  1.69it/s]

In [12]:
dataset_traducido = pd.read_parquet('/Volumes/data_ml4ds/IntelComp/Datasets/tecnoempleo/translation/tecnoempleo_translated.parquet')

In [13]:
dataset_traducido

,id,title_translated,professionalSkills_translated,description_translated
0,0000pe9afx742e1452ge,Administrador Wintel Inglés alto,Administrador,KRELL CONSULTINGampTRAINING es una compañía de...
0,0001ia104ed981f901md,Ingeniero sénior de control de calidad,Ensayador,"En nombre de Lotoland, Ciklum está buscando un..."
0,0001oc82e7b9244a3doc,Desarrollo SW C,,Necesitamos incorporar con la mayor brevedad p...
0,0001ub5d21fd3r3d5ba9,Analista Programador Senior LAMP,Programador Analista,En Serem estamos comprometidos con diversos pr...
0,0002474f2l1c4384b0we,Soporte técnico Usuarios Media Journey,Soporte técnico de la mesa de ayuda,Sermicro es una empresa Integradora de referen...
...,...,...,...,...
0,2053w75b8x08746c050f,Programador informático,Analista Analista Programador Desarrollador Web,Empresa internacional residente en Tenerife bu...
0,20560e4demfc2yd431i6,Director de tecnología,,Qué necesitarás para encajar con el puesto T...
0,2056gb358kd514c1d6g8,Programador Java Senior,Programador,Nos encontramos en búsqueda de un Programador ...
0,2057k81c1a725k210d60,ingeniero de datos,Analista,Actualmente seleccionamos para incorporación a...
